In [1]:
# set env via .env file
import sys
import pandas as pd
import datetime
import time
import munch
import shared
import config
import model
import provider_yfinance as provider

In [2]:
print('init> initialize environment')
config.print_env()

init> initialize environment
TF version: 2.1.0
Keras version: 2.2.4-tf
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is there a GPU available: False
Is the Tensor on GPU #0: False
Device name: /job:localhost/replica:0/task:0/device:CPU:0
Eager Execution enabled: True


In [3]:
cfg = config.get_config('^GDAXI', True)
#config.overwrite_end_dt(cfg, '2020-01-16')
#config.save_config(cfg)

config> current directory:/mnt/c/notebooks/sandbox
config> data period: from '2018-02-25' to '2020-01-26'
config> download period: from '2017-12-10' to '2020-01-26'
shared> parsing stocks from web '^GDAXI' ...
config> benchmarks: '['000001.SS', 'AUDUSD=X', 'BTCUSD=X', 'CNY=X', 'ETHUSD=X', 'EURCAD=X', 'EURCHF=X', 'EURGBP=X', 'EURHUF=X', 'EURJPY=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBPJPY=X', 'GBPUSD=X', 'GC=F', 'HKD=X', 'IDR=X', 'IMOEX.ME', 'INR=X', 'JPY=X', 'KW=F', 'LC=F', 'LH=F', 'MXN=X', 'MYR=X', 'NZDUSD=X', 'PA=F', 'PHP=X', 'PL=F', 'RUB=X', 'S=F', 'SGD=X', 'SI=F', 'SM=F', 'THB=X', 'ZAR=X', '^AORD', '^AXJO', '^BFX', '^BSESN', '^BUK100P', '^BVSP', '^DJI', '^FCHI', '^FTSE', '^GDAXI', '^GSPC', '^GSPTSE', '^HSI', '^IPSA', '^IXIC', '^JKSE', '^JN0U.JO', '^KLSE', '^KS11', '^MERV', '^MXX', '^N100', '^N225', '^NYA', '^NZ50', '^RUT', '^STI', '^STOXX50E', '^TA125.TA', '^TWII', '^VIX', '^XAX']'
config> stocks: '['1COV.DE', 'ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BEI.DE', 'BMW.DE', 'CON.DE'

In [4]:
print(f'download> start downloading data {cfg.prepare.data_end_dt} ...')
start_ts = time.time()

download> start downloading data 2020-01-26 ...


In [5]:
%%time
# total: 17 s
# compact: 1m

cfg_stocks, data_stocks = provider.load_stocks(cfg, compact=True)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126/stocks_1d.pkl'
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126/stocks_1d_meta.pkl'
tickers-stocks> downloading histories ...
[*********************100%***********************]  30 of 30 completed
WARN: tickers-stocks> #missing: 35695
ticker-1COV.DE> loading ticker data ...
ticker-ADS.DE> loading ticker data ...
ticker-ALV.DE> loading ticker data ...
ticker-BAS.DE> loading ticker data ...
ticker-BAYN.DE> loading ticker data ...
ticker-BEI.DE> loading ticker data ...
WARN: ticker-BEI.DE> #missing: 10
ticker-BMW.DE> loading ticker data ...
ticker-CON.DE> loading ticker data ...
yf-cashflowStatementHistory> internal error: Internal Server Error
yf-cashflowStatementHistoryQuarterly> internal error: Internal Server Error
yf-balanceSheetHistory> internal error: Internal Server Error
yf-balanceSheetHistoryQuarterly> internal error: Internal Server Error
yf-incomeStatementHistory> internal 

In [6]:
%%time
# total (download): 1h
# total (cached): 25 s
# compact: 3m

cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg, compact=True)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126/benchmarks_1d.pkl'
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126/benchmarks_1d_meta.pkl'
tickers-benchmarks> downloading histories ...
[*********************100%***********************]  68 of 68 completed
WARN: tickers-benchmarks> #missing: 196805
ticker-000001.SS> loading ticker data ...
ticker-AUDUSD=X> loading ticker data ...
ticker-BTCUSD=X> loading ticker data ...
ticker-CNY=X> loading ticker data ...
ticker-ETHUSD=X> loading ticker data ...
ticker-EURCAD=X> loading ticker data ...
ticker-EURCHF=X> loading ticker data ...
ticker-EURGBP=X> loading ticker data ...
ticker-EURHUF=X> loading ticker data ...
ticker-EURJPY=X> loading ticker data ...
ticker-EURJPY=X> loading ticker data ...
ticker-EURSEK=X> loading ticker data ...
ticker-EURUSD=X> loading ticker data ...
ticker-GBPJPY=X> loading ticker data ...
ticker-GBPUSD=X> loading ticker data ...
ticker-GC=F> loading ticker da

In [7]:
duration = time.time() - start_ts
print(f'download> download finished, duration: {duration:.2f} s')

download> download finished, duration: 188.25 s


In [8]:
start_ts = time.time()
print(f'prepare> preparing data ...')

prepare> preparing data ...


In [9]:
%%time 
# 344 ms
cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

CPU times: user 188 ms, sys: 62.5 ms, total: 250 ms
Wall time: 261 ms


In [10]:
%%time 
# total: 2.55 s
prep_stocks = provider.prepare_stocks(cfg, data_stocks)
prep_benchmarks = provider.prepare_benchmarks(cfg, data_benchmarks)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126//stocks_prep.pkl'
shared> prepare stock '1COV.DE' from '2018-02-25' (avail: '2015-10-06') to '2020-01-26'
shared> prepare stock 'ADS.DE' from '2018-02-25' (avail: '1998-06-24') to '2020-01-26'
shared> prepare stock 'ALV.DE' from '2018-02-25' (avail: '1996-12-16') to '2020-01-26'
shared> prepare stock 'BAS.DE' from '2018-02-25' (avail: '1996-12-16') to '2020-01-26'
shared> prepare stock 'BAYN.DE' from '2018-02-25' (avail: '1996-12-16') to '2020-01-26'
shared> prepare stock 'BEI.DE' from '2018-02-25' (avail: '1998-07-22') to '2020-01-26'
shared> prepare stock 'BMW.DE' from '2018-02-25' (avail: '1996-11-08') to '2020-01-26'
shared> prepare stock 'CON.DE' from '2018-02-25' (avail: '1996-12-16') to '2020-01-26'
shared> prepare stock 'DAI.DE' from '2018-02-25' (avail: '1996-10-30') to '2020-01-26'
shared> prepare stock 'DB1.DE' from '2018-02-25' (avail: '2001-02-05') to '2020-01-26'
shared> prepare stock 'DBK.DE' fr

In [11]:
%%time
# total: 1.2 s
enc_stocks = provider.encode_stocks(cfg, prep_stocks)
enc_benchmarks = provider.encode_benchmarks(cfg, prep_benchmarks, prep_stocks)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126//stocks_enc.pkl'
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126//benchmarks_enc.pkl'
CPU times: user 938 ms, sys: 78.1 ms, total: 1.02 s
Wall time: 974 ms


In [12]:
%%time 
# total: 20min - 50min
# compact: 20min

for submodel_settings in cfg.train.settings:
    print(f"sm-{submodel_settings.id}> preparing submodel data ...")
    model_data = provider.prepare_submodel_data(cfg, submodel_settings, enc_stocks, enc_benchmarks)
    # update num_features setting (informational)
    submodel_settings.num_features = len(model_data.X[0][0][0][0])
config.save_config(cfg)

sm-lookback_3-label_1> preparing submodel data ...
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200126/lookback_3-label_1/submodel_data.pkl'
sm-lookback_3-label_1> generating relative benchmarks data ...
sm-lookback_3-label_1> preparing stock: '1COV.DE' ...


Using TensorFlow backend.


sm-lookback_3-label_1> preparing stock: 'ADS.DE' ...
sm-lookback_3-label_1> preparing stock: 'ALV.DE' ...
sm-lookback_3-label_1> preparing stock: 'BAS.DE' ...
sm-lookback_3-label_1> preparing stock: 'BAYN.DE' ...
sm-lookback_3-label_1> preparing stock: 'BEI.DE' ...
sm-lookback_3-label_1> preparing stock: 'BMW.DE' ...
sm-lookback_3-label_1> preparing stock: 'CON.DE' ...
sm-lookback_3-label_1> preparing stock: 'DAI.DE' ...
sm-lookback_3-label_1> preparing stock: 'DB1.DE' ...
sm-lookback_3-label_1> preparing stock: 'DBK.DE' ...
sm-lookback_3-label_1> preparing stock: 'DPW.DE' ...
sm-lookback_3-label_1> preparing stock: 'DTE.DE' ...
sm-lookback_3-label_1> preparing stock: 'EOAN.DE' ...
WARN seq-1> no further previous year data available!
WARN seq-2> no further previous year data available!
WARN seq-3> no further previous year data available!
WARN seq-4> no further previous year data available!
WARN seq-5> no further previous year data available!
WARN seq-6> no further previous year data av

In [13]:
duration = time.time() - start_ts
print(f'prepare> preparation finished, duration: {duration:.2f} s')

prepare> preparation finished, duration: 1019.60 s


In [14]:
start_ts = time.time()
print(f'train> training started ...')

train> training started ...


In [15]:
[(i, s.id) for i, s in enumerate(cfg.train.settings)]

[(0, 'lookback_3-label_1'),
 (1, 'lookback_3-label_2'),
 (2, 'lookback_5-label_1'),
 (3, 'lookback_21-label_5'),
 (4, 'lookback_100-label_7')]

In [ ]:
%%time

model.train_full(cfg, 0)

In [ ]:
duration = time.time() - start_ts
print(f'train> training finished, duration: {duration:.2f} s')

In [ ]:
start_ts = time.time()
print(f'predict> prediction started ...')